In [310]:
import pandas as pd 
import numpy as np
import altair as alt
import re
import pickle

In [311]:
# Import and clean the given data, check for duplicates
forbes_billionaires = pd.read_csv("forbes_billionaires (2).csv")
IsDuplicate = forbes_billionaires["Name"].duplicated()
DuplicatedNames = forbes_billionaires[IsDuplicate]
DuplicatedNamesCounts = forbes_billionaires["Name"].value_counts()[DuplicatedNames["Name"]]
print("Number of duplicate name entries: ", len(DuplicatedNames))
print(DuplicatedNamesCounts)

Number of duplicate name entries:  3
Robert Miller            2
Wang Yanqing & family    2
Li Li                    2
Name: Name, dtype: int64


In [312]:
# Manually inspect the three duplicate entries, verify their information, and individually address each duplicate

# Robert Miller
print(forbes_billionaires[forbes_billionaires["Name"] == "Robert Miller"])

               Name  NetWorth         Country                  Source  Rank  \
1331  Robert Miller       2.4  United Kingdom                  retail  1299   
1625  Robert Miller       2.0          Canada  electronics components  1580   

       Age         Residence Citizenship    Status  Children  Self_made  \
1331  75.0  Montreal, Canada      Canada  Divorced       2.0        1.0   
1625  75.0  Montreal, Canada      Canada  Divorced       2.0        1.0   

                     Education         University  
1331  Bachelor of Arts/Science   Rider University  
1625  Bachelor of Arts/Science   Rider University  


In [313]:
# A cursory search to confirm data validity reveals the existence of both a Robert Miller of Canada and A Robert Miller of the United Kingdom.
# Modifying entry 1331 to match the 2021 profile sourced from https://www.forbes.com/profile/robert-miller/?sh=2380a72252e1
forbes_billionaires.at[1331,"Age"] = 87
forbes_billionaires.at[1331,"Residence"] = "Hong Kong, Hong Kong"
forbes_billionaires.at[1331,"Citizenship"] = "United Kingdom"
forbes_billionaires.at[1331,"Status"] = "Married"
forbes_billionaires.at[1331,"Children"] = 3
forbes_billionaires.at[1331,"University"] = "Cornell University"

print(forbes_billionaires[forbes_billionaires["Name"] == "Robert Miller"])

               Name  NetWorth         Country                  Source  Rank  \
1331  Robert Miller       2.4  United Kingdom                  retail  1299   
1625  Robert Miller       2.0          Canada  electronics components  1580   

       Age             Residence     Citizenship    Status  Children  \
1331  87.0  Hong Kong, Hong Kong  United Kingdom   Married       3.0   
1625  75.0      Montreal, Canada          Canada  Divorced       2.0   

      Self_made                 Education          University  
1331        1.0  Bachelor of Arts/Science  Cornell University  
1625        1.0  Bachelor of Arts/Science    Rider University  


In [314]:
# Wang Yanqing & family
print(forbes_billionaires[forbes_billionaires["Name"] == "Wang Yanqing & family"])

                       Name  NetWorth Country                 Source  Rank  \
700   Wang Yanqing & family       4.2   China   electrical equipment   680   
1919  Wang Yanqing & family       1.7   China  carbon fiber products  1833   

       Age    Residence Citizenship   Status  Children  Self_made Education  \
700   55.0  Wuxi, China       China  Married       NaN        1.0       NaN   
1919  55.0  Wuxi, China       China  Married       NaN        1.0       NaN   

     University  
700         NaN  
1919        NaN  


In [315]:
# Similar to Robert Miller, two profiles of "Wang Yanqing & family" exist: one from Wuxi, China, and one from Weihai, China.
# Modifying entry 1919 to match the 2021 profile sourced from https://www.forbes.com/profile/wang-yanqing-1/?sh=59f80db94910
forbes_billionaires.at[1919,"Age"] = 74
forbes_billionaires.at[1919,"Residence"] = "Weihai, China"
forbes_billionaires.at[1919,"Citizenship"] = "China"
forbes_billionaires.at[1919,"Status"] = "Married"
forbes_billionaires.at[1919,"Self_made"] = 0
print(forbes_billionaires[forbes_billionaires["Name"] == "Wang Yanqing & family"])

                       Name  NetWorth Country                 Source  Rank  \
700   Wang Yanqing & family       4.2   China   electrical equipment   680   
1919  Wang Yanqing & family       1.7   China  carbon fiber products  1833   

       Age      Residence Citizenship   Status  Children  Self_made Education  \
700   55.0    Wuxi, China       China  Married       NaN        1.0       NaN   
1919  74.0  Weihai, China       China  Married       NaN        0.0       NaN   

     University  
700         NaN  
1919        NaN  


In [316]:
# Li Li
print(forbes_billionaires[forbes_billionaires["Name"] == "Li Li"])

       Name  NetWorth Country           Source  Rank   Age        Residence  \
693   Li Li       4.2   China       healthcare   680  57.0  Shenzhen, China   
1976  Li Li       1.6   China  pharmaceuticals  1931  57.0  Shenzhen, China   

     Citizenship   Status  Children  Self_made                 Education  \
693        China  Married       NaN        1.0  Bachelor of Arts/Science   
1976       China  Married       NaN        1.0  Bachelor of Arts/Science   

               University  
693    Sichuan University  
1976   Sichuan University  


In [317]:
# Once again, a cursory search reveals two separate "Li Li"s of Shenzhen, China and Changsha, China.
# Modifying entry 693 to match the 2021 profile sourced from https://www.forbes.com/profile/li-li-3/?sh=2abdfdcc782c
forbes_billionaires.at[693,"Age"] = 55
forbes_billionaires.at[693,"Residence"] = "Changsha, China"
forbes_billionaires.at[693,"Citizenship"] = "China"
forbes_billionaires.at[693,"Status"] = np.nan
forbes_billionaires.at[693,"Education"] = np.nan
forbes_billionaires.at[693,"University"] = np.nan
print(forbes_billionaires[forbes_billionaires["Name"] == "Li Li"])

       Name  NetWorth Country           Source  Rank   Age        Residence  \
693   Li Li       4.2   China       healthcare   680  55.0  Changsha, China   
1976  Li Li       1.6   China  pharmaceuticals  1931  57.0  Shenzhen, China   

     Citizenship   Status  Children  Self_made                 Education  \
693        China      NaN       NaN        1.0                       NaN   
1976       China  Married       NaN        1.0  Bachelor of Arts/Science   

               University  
693                   NaN  
1976   Sichuan University  


In [318]:
# Modify Education and University columns to note heighest attainment, separate Bachelors and Masters universities
print("Number of entries without Education data: ", sum(forbes_billionaires["Education"].isna()))
print("Number of entries with Education data: ", sum(forbes_billionaires["Education"].notna()))
# Copy Education into HEA (Highest Educational Attainment)
forbes_billionaires["HEA"] = forbes_billionaires["Education"]
# Concatenate Education and University columns
forbes_billionaires["Education"] = forbes_billionaires.apply(lambda x:'%s,%s' % (x["Education"],x["University"]) if pd.notna(x["Education"]) else np.nan, axis=1)
#forbes_billionaires["Education"].split(";")

Number of entries without Education data:  1347
Number of entries with Education data:  1408


In [319]:
# Show unique values in HEA
forbes_billionaires["HEA"].value_counts()

Bachelor of Arts/Science                 637
Master of Business Administration        166
Master of Science                         82
Drop Out                                  74
Bachelor of Science                       46
                                        ... 
Certificate                                1
Bachelor of Social Sciences                1
Jiangsu Province Wujin Teacher School      1
Chinese Culture University                 1
High School                                1
Name: HEA, Length: 66, dtype: int64

In [320]:
# Correct naming for University of California
#IsUC = forbes_billionaires["Education"].str.contains('University of California, ', na=False, flags=re.IGNORECASE, regex=True)
#forbes_billionaires[IsUC] = forbes_billionaires[IsUC].Education.replace("University of California, ", "UC ", inplace=True, regex=True)


In [321]:
# High School Diploma holders and dropouts
IsHS = forbes_billionaires["Education"].str.contains('High School', na=False, flags=re.IGNORECASE, regex=True)
print("Number of entries with a High School HEA/Dropout: ", sum(IsHS))
#forbes_billionaires.loc[IsHS]

Number of entries with a High School HEA/Dropout:  43


In [322]:
# Bachelors Degree Holders, and their University
IsBachelor = forbes_billionaires["Education"].str.contains('Bachelor', na=False, flags=re.IGNORECASE, regex=True)
print("Number of entries with a Bachelor's Degree: ", sum(IsBachelor))
Bachelors = forbes_billionaires[IsBachelor]
BachelorUniversities = Bachelors["Education"].str.split(pat=";").apply(lambda i: [(lambda x: x.split(",")[-1] if "bachelor" in x.casefold() else np.nan)(x) for x in i][0] if type(i) is list else np.nan)
BachelorUniversitiesCounts = BachelorUniversities.value_counts()
BachelorUniversitiesSelfmadeCounts = BachelorUniversities[Bachelors["Self_made"] == 1].value_counts()
BachelorUniversitiesInheritedCounts = BachelorUniversities[Bachelors["Self_made"] == 0].value_counts()
#print(BachelorUniversitiesCounts)

Number of entries with a Bachelor's Degree:  1007


In [323]:
# Bachelors drop outs, and their University
IsDropOut = forbes_billionaires["Education"].str.contains('Drop Out', na=False, flags=re.IGNORECASE, regex=True)
IsBachelorDropOut = IsDropOut^(IsDropOut & (IsBachelor | IsHS))
print("Number of entries dropped out of Bachelors", sum(IsBachelorDropOut))
BachelorDropOuts = forbes_billionaires[IsBachelorDropOut]
BachelorDropOutUniversities = BachelorDropOuts["Education"].str.split(pat=";").apply(lambda i: [(lambda x: x.split(",")[-1] if "drop out" in x.casefold() else np.nan)(x) for x in i][0] if type(i) is list else np.nan)
BachelorDropOutUniversitiesCounts = BachelorDropOutUniversities.value_counts()
BachelorDropOutUniversitiesSelfmadeCounts = BachelorDropOutUniversities[BachelorDropOuts["Self_made"] == 1].value_counts()
BachelorDropOutUniversitiesInheritedCounts = BachelorDropOutUniversities[BachelorDropOuts["Self_made"] == 0].value_counts()
#print(BachelorDropOutUniversitiesCounts)

Number of entries dropped out of Bachelors 63


In [324]:
# Masters/MBA Degree Holders, and their University
IsMaster = forbes_billionaires["Education"].str.contains('Master', na=False, flags=re.IGNORECASE, regex=True)
print("Number of entries with a Master's Degree: ", sum(IsMaster))
#forbes_billionaires.[IsMaster]

Number of entries with a Master's Degree:  473


In [325]:
# Doctorate Degree Holders, and their University
IsDoctor = forbes_billionaires["Education"].str.contains('Doctor', na=False, flags=re.IGNORECASE, regex=True)
print("Number of entries with a Doctorate Degree: ", sum(IsDoctor))
#forbes_billionaires.[IsDoctor]

Number of entries with a Doctorate Degree:  132


In [326]:
alt.Chart(forbes_billionaires).mark_bar().encode(alt.X("NetWorth", bin=True), y='count()',)

d:\Anaconda\envs\streamlit\Lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [327]:
alt.Chart(forbes_billionaires[IsHS]).mark_bar().encode(alt.X("NetWorth", bin=True), y='count()',)

alt.Chart(...)

In [328]:
alt.Chart(forbes_billionaires[IsBachelor]).mark_bar().encode(alt.X("NetWorth", bin=True), y='count()',)

alt.Chart(...)

In [329]:
alt.Chart(forbes_billionaires[IsMaster]).mark_bar().encode(alt.X("NetWorth", bin=True), y='count()',)

alt.Chart(...)

In [330]:
alt.Chart(forbes_billionaires[IsDoctor]).mark_bar().encode(alt.X("NetWorth", bin=True), y='count()',)

alt.Chart(...)

**Universities ranked by the number of billionaires who studied there (begin with bachelors)**
angle: show raw universities that billionaires studied at (BOTH graduate and dropout)
- split graduates and dropout
- split affluent background vs self-made (on the assumption that each billionaire made their wealth after their bachelors)
- bonus angle: top 10 (or more) universities and their annual tuition fees before financial aid
- other bonus angle: billionaires by net worth and educational attainment (high school dropout, college dropout, bachelors graduate, masters degree/MBA)

In [331]:
# Plotting DataFrame definitions
BachelorsCountsDF = pd.DataFrame({
    'index': BachelorUniversitiesCounts.index, 
    'values': BachelorUniversitiesCounts.values, 
    'label':"Bachelor's degree holders"
})
BachelorDropOutsCountsDF = pd.DataFrame({
    'index': BachelorDropOutUniversitiesCounts.index, 
    'values': BachelorDropOutUniversitiesCounts.values, 
    'label':"Bachelor's drop outs"
})
BachelorsSelfmadeCountsDF = pd.DataFrame({
    'index': BachelorUniversitiesSelfmadeCounts.index, 
    'values': BachelorUniversitiesSelfmadeCounts.values, 
    'Self_made': True, 
    'Drop_out': False, 
    'label':"Self-made Bachelor's degree holders"
})
BachelorDropOutsSelfmadeCountsDF = pd.DataFrame({
    'index': BachelorDropOutUniversitiesSelfmadeCounts.index, 
    'values': BachelorDropOutUniversitiesSelfmadeCounts.values, 
    'Self_made': True, 
    'Drop_out': True, 
    'label':"Self-made Bachelor's drop outs"
})
BachelorsInheritedCountsDF = pd.DataFrame({
    'index': BachelorUniversitiesInheritedCounts.index, 
    'values': BachelorUniversitiesInheritedCounts.values, 
    'Self_made': False, 
    'Drop_out': False, 
    'label':"Inherited Bachelor's degree holders"
})
BachelorDropOutsInheritedCountsDF = pd.DataFrame({
    'index': BachelorDropOutUniversitiesInheritedCounts.index, 
    'values': BachelorDropOutUniversitiesInheritedCounts.values, 
    'Self_made': False, 
    'Drop_out': True, 
    'label':"Inherited Bachelor's drop outs"
})
# BachelorUniversitiesDF = pd.DataFrame({
#     'University': BachelorUniversities.values,
#     'Self-made': Bachelors['Self_made']
# })
# BachelorDropOutUniversitiesDF = pd.DataFrame({
#     'University': BachelorDropOutUniversities.values,
#     'Self-made': BachelorDropOuts['Self_made']
# })

PlottingDFs = {
    "BachelorSelfmade": BachelorsSelfmadeCountsDF,
    "BachelorDOSelfmade": BachelorDropOutsSelfmadeCountsDF,
    "BachelorInherited": BachelorsInheritedCountsDF,
    "BachelorDOInherited": BachelorDropOutsInheritedCountsDF
}

In [332]:
# Chart of Bachelors graduates
alt.Chart(BachelorsCountsDF).mark_bar().encode(
    alt.X('index', sort='-y'), 
    alt.Y('values'))

d:\Anaconda\envs\streamlit\Lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [333]:
# Stacked Chart of Bachelors graduates and dropouts

alt.Chart(pd.concat([BachelorsCountsDF, BachelorDropOutsCountsDF])).mark_bar().encode(
    alt.X('index', sort='-y'), 
    alt.Y('values'),
    color='label')

alt.Chart(...)

In [334]:
# Stacked Chart of Bachelors graduates and dropOuts, whether Selfmade or Inherited

alt.Chart(pd.concat([PlottingDFs["BachelorSelfmade"], PlottingDFs["BachelorDOSelfmade"], PlottingDFs["BachelorInherited"], PlottingDFs["BachelorDOInherited"]])).mark_bar().encode(
    alt.X('index', sort='-y'), 
    alt.Y('values'),
    color='label')

alt.Chart(...)

In [335]:
# Export the preprocessed dataframe for use in the Streamlit dashboard
forbes_billionaires.to_pickle("./forbes_billionaires_preprocessed.pkl")
with open("./PlottingDFs.pkl", "wb") as handle:
    pickle.dump(PlottingDFs, handle, protocol=pickle.HIGHEST_PROTOCOL)